# 1.連結至TWSE, Get 抽籤資訊
# 2.符合張數>1000, 獲利超過10%
# 3.利用Line 通知


In [9]:
import pandas as pd
# 連至TWSE 爬蟲
url='https://www.twse.com.tw/announcement/publicForm?response=html&yy=2022'
list_str = pd.read_html(url)  #list
df=list_str[0]
#df

In [2]:
df2=df.copy()

In [3]:
row_len,col_len=df2.shape

In [1]:
import datetime
Date_Condition=datetime.date.today()

list_data=[]

In [3]:
print(Date_Condition)
print(type(Date_Condition))


2022-10-19
<class 'datetime.date'>


In [6]:
import datetime

x = datetime.date(2021, 10, 10)

print(x)
print(type(x))

2021-10-10
<class 'datetime.date'>


In [5]:
for m in range(row_len): #Row index 每次跑一筆Data
    date_arr=df2.iloc[m, 6].split('/')
    input_date = datetime.date(int(date_arr[0])+1911, int(date_arr[1]), int(date_arr[2]))
    #print(df2.iloc[m, 6])
    #if df2.iloc[m,6]             #申購結束日
    if Date_Condition >= input_date :
        break
           



In [6]:
df2=df2.drop(list(range(m,row_len)))

In [7]:
#df.columns
list_columns=[]


#df.columns
#for n in range(df.columns.size):  #Column index 每次跑一個
for n in range(col_len):  # Column index 每次跑一個
    list_columns.append(df2.columns[n][1])
    # print(df2.columns[n][1])


In [8]:
df2.columns=list_columns

In [16]:
df2.head(0)

,序號,抽籤日期,證券名稱,證券代號,發行市場,申購開始日,申購結束日,承銷股數,實際承銷股數,承銷價(元),實際承銷價(元),撥券日期(上市、上櫃日期),主辦券商,申購股數,總承銷金額(元),總合格件,中籤率(%),取消公開抽籤


In [17]:
msg=df2.columns[0]+df2.columns[1]+df2.columns[2]
msg

'序號抽籤日期證券名稱'

In [123]:
print(m)

6


In [139]:
#type(df2.iloc[1, 1])
type(df2.columns[1])
message=df2.columns[1]+'-'+df2.iloc[1, 1]

In [9]:
df2=df2.set_index('序號')

In [10]:
df2.to_excel(r'D:\temp\out.xlsx') 

# Get 股價資訊

## 需裝 twstock

In [11]:
import twstock
print(twstock.codes['6855'])

KeyError: '6855'

In [12]:
stock = twstock.realtime.get('6855')  # 查詢上市股票之即時資料
stock

{'msgArray': [],
 'referer': '',
 'userDelay': 5000,
 'rtcode': '5001',
 'queryTime': {'sysDate': '20220924',
  'stockInfoItem': 448,
  'stockInfo': 61641,
  'sessionStr': 'UserSession',
  'sysTime': '13:45:10',
  'showChart': False,
  'sessionFromTime': -1,
  'sessionLatestTime': -1},
 'rtmessage': 'Empty Query.',
 'success': False}

In [13]:
latest_trade_price=stock['realtime']['latest_trade_price']
latest_trade_price

KeyError: 'realtime'

# 處理Line 問題 

## 1.申請Line Notify(https://notify-bot.line.me/zh_TW/)

## 2.Get 權杖
### VZSWE7IPJ1NTiTZ9lAcHUUvHBvHLC3yKbHb3JpMNG5E

## 3.發送Line Notify 通知

In [35]:
import requests
import twstock
import time


def lineNotify(token, msg):
    headers = {
        "Authorization": "Bearer " + token,  # 注意"Bearer ", 有一個空白
        "Content-Type": "application/x-www-form-urlencoded"
    }

    payload = {'message': msg}
    notify = requests.post("https://notify-api.line.me/api/notify", headers=headers, params=payload)
    return notify.status_code


row_len, col_len = df2.shape
token = 'VZSWE7IPJ1NTiTZ9lAcHUUvHBvHLC3yKbHb3JpMNG5E'  # 權杖

for m in range(row_len):  # Row index 每次跑一筆Data
    #    print(df2.iloc[m, 1]+','+df2.iloc[m, 2]+','+df2.iloc[m, 4]+','+df2.iloc[m, 5])
    #    print(df2.columns[1]+','+df2.columns[2]+','+df2.columns[4]+','+df2.columns[5])

    # df2.iloc[m, 1] #證券名稱
    # df2.iloc[m, 2] #證券代號
    # df2.iloc[m, 4] #申購開始日
    # df2.iloc[m, 5] #申購結束日
    # df2.iloc[m, 7] #實際承銷股數
    # df2.iloc[m, 9] #實際承銷價(元)
    message = df2.columns[1] + '=' + df2.iloc[m, 1] + ',' + \
              df2.columns[2] + '=' + df2.iloc[m, 2] + ',' + \
              df2.columns[4] + '=' + df2.iloc[m, 4] + ',' + \
              df2.columns[5] + '=' + df2.iloc[m, 5] + ',' + \
              df2.columns[7] + '=' + str(df2.iloc[m, 7]) + ',' + \
              df2.columns[9] + '=' + str(df2.iloc[m, 9])

    latest_trade_price = ''
    price_gap = ''

    while True :
        time.sleep(5)
        stock = twstock.realtime.get(df2.iloc[m, 2])  # 查詢上市股票之即時資料
        if stock['success']:  # 上市股票
            latest_trade_price = stock['realtime']['latest_trade_price']
            if latest_trade_price != '-' :
                # print(stock)
                # print('latest_trade_price=' + latest_trade_price)
                price_gap = str(eval(latest_trade_price + '-' + str(df2.iloc[m, 9])))  # 價差
                underwriting_price = float(df2.iloc[m, 9])  # 承銷價
                Premium_ratio = (float(price_gap) / underwriting_price) * 100  # 溢價比
                # message = message + ' ,最新價格=' + latest_trade_price + ' ,價差=' + price_gap + ' ,溢價差(%)=' + str(Premium_ratio)
                message = message + ' ,最新價格=' + latest_trade_price + ' ,價差=' + price_gap + ' ,溢價差(%)=' + '{:3.2f}'.format(Premium_ratio)

                
                break
        else :
            break

    # print(message+'\n')

    code = lineNotify(token, message)
    if code == 200:
        print('發送 Line Notify 成功')
    else:
        print('發送 Line Notify 失敗')

發送 Line Notify 成功
發送 Line Notify 成功
發送 Line Notify 成功
發送 Line Notify 成功
發送 Line Notify 成功
發送 Line Notify 成功
發送 Line Notify 成功


## 以下是測試

In [46]:
df2.iloc[1, 9]

100.0

In [ ]:
import requests

msg = '這是 Line Notify 發送的訊息'
token = 'VZSWE7IPJ1NTiTZ9lAcHUUvHBvHLC3yKbHb3JpMNG5E' #權杖
headers={
    "Authorization": "Bearer " + token,  #注意"Bearer ", 有一個空白
    "Content-Type": "application/x-www-form-urlencoded"
}

payload={'message':msg}
notify=requests.post("https://notify-api.line.me/api/notify",headers=headers,params=payload)
if notify.status_code==200:
    print('發送 Line Notify 成功')
else:
    print('發送 Line Notify 失敗')

In [5]:
import datetime
now=datetime.datetime.now()
now

datetime.datetime(2022, 9, 21, 11, 19, 23, 326269)



# 以下是以前寫法-20220918





In [14]:
df.columns

MultiIndex([('公開申購公告-抽籤日程表',            '序號'),
            ('公開申購公告-抽籤日程表',          '抽籤日期'),
            ('公開申購公告-抽籤日程表',          '證券名稱'),
            ('公開申購公告-抽籤日程表',          '證券代號'),
            ('公開申購公告-抽籤日程表',          '發行市場'),
            ('公開申購公告-抽籤日程表',         '申購開始日'),
            ('公開申購公告-抽籤日程表',         '申購結束日'),
            ('公開申購公告-抽籤日程表',          '承銷股數'),
            ('公開申購公告-抽籤日程表',        '實際承銷股數'),
            ('公開申購公告-抽籤日程表',        '承銷價(元)'),
            ('公開申購公告-抽籤日程表',      '實際承銷價(元)'),
            ('公開申購公告-抽籤日程表', '撥券日期(上市、上櫃日期)'),
            ('公開申購公告-抽籤日程表',          '主辦券商'),
            ('公開申購公告-抽籤日程表',          '申購股數'),
            ('公開申購公告-抽籤日程表',      '總承銷金額(元)'),
            ('公開申購公告-抽籤日程表',          '總合格件'),
            ('公開申購公告-抽籤日程表',        '中籤率(%)'),
            ('公開申購公告-抽籤日程表',        '取消公開抽籤')],
           )

In [47]:
df.to_excel('c:\out.xlsx')  

In [18]:
print(type(df.columns[1]))

<class 'tuple'>


In [19]:
df.columns[1][1]

'抽籤日期'

In [23]:
df.index[1]

1

In [3]:
#df.columns
list_columns=[]

row_len,col_len=df.shape

#df.columns
#for n in range(df.columns.size):  #Column index 每次跑一個
for n in range(col_len):  # Column index 每次跑一個
    list_columns.append(df.columns[n][1])
    print(df.columns[n][1])


序號
抽籤日期
證券名稱
證券代號
發行市場
申購開始日
申購結束日
承銷股數
實際承銷股數
承銷價(元)
實際承銷價(元)
撥券日期(上市、上櫃日期)
主辦券商
申購股數
總承銷金額(元)
總合格件
中籤率(%)
取消公開抽籤


In [4]:
print(list_columns)

['序號', '抽籤日期', '證券名稱', '證券代號', '發行市場', '申購開始日', '申購結束日', '承銷股數', '實際承銷股數', '承銷價(元)', '實際承銷價(元)', '撥券日期(上市、上櫃日期)', '主辦券商', '申購股數', '總承銷金額(元)', '總合格件', '中籤率(%)', '取消公開抽籤']


In [5]:

#print(list_columns)
#df.iloc[0,:]  #第一row資料
import datetime
Date_Condition=datetime.date.today()

list_data=[]

In [12]:
#for m in range(df.shape[0]): #Row index 每次跑一筆Data
for m in range(row_len): #Row index 每次跑一筆Data
    date_arr=df.iloc[m, 6].split('/')
    input_date = datetime.date(int(date_arr[0])+1911, int(date_arr[1]), int(date_arr[2]))
    #print(df.iloc[m, 6])
    #if df.iloc[m,6]             #申購結束日
    if Date_Condition < input_date :
        list_data.append(df.iloc[m,:])
        print(df.iloc[m,:])
    else :
        break


print(list_data)

公開申購公告-抽籤日程表  證券名稱              111央債甲09
              證券代號                A111ID
              發行市場                中央登錄公債
              申購開始日            111/09/19
              申購結束日            111/09/23
              承銷股數                900000
              實際承銷股數              900000
              承銷價(元)               100.0
              實際承銷價(元)             100.0
              撥券日期(上市、上櫃日期)    111/09/30
              主辦券商                中華郵政公司
              申購股數                 10000
              總承銷金額(元)          90000000
              總合格件                     0
              中籤率(%)                 0.0
              取消公開抽籤                 NaN
Name: 0, dtype: object
公開申購公告-抽籤日程表  證券名稱              111央債甲09
              證券代號                A111IA
              發行市場                中央登錄公債
              申購開始日            111/09/19
              申購結束日            111/09/23
              承銷股數                120000
              實際承銷股數              120000
              承銷價(元)              

In [47]:
500 // 3

166

In [49]:
def calc_power(a,b):
        return a**b
base=input('Enter the number for the base')
exponent=input('Enter th number for th exponent:')
result=calc_power(base,exponent)
print("The result is"+result)

Enter the number for the base2
Enter th number for th exponent:3


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'str'

In [51]:
a='con1'
b=a
a+='con2'
print(a)
print(b)

con1con2
con1
